# Algoritmusok elemző összehasonlítása

Limbek Zsófia munkája

## 1. Bevezetés

Dolgozatomban az Eszterházy Károly Egyetem LBT_PI212G3 kódú, Adatszerkezetek és algoritmusok gy. tárgyából tanult algoritmusokat elemzem. Az implementációk mellett különböző bemenetekkel mért futásidőket is bemutatok, illetve összehasonlítom az azonos célra használt különböző algoritmusokat.

Az algoritmusok implementálásához, a futásidők méréséhez, az adatok összegyűjtéséhez és ábrázolásához a `Python 3` programnyelvet használtam (verzió: `3.8`). A teljes kód megtalálható a [ebben](https://github.com/zslim/algoritmusok_elemzes) a GitHub repóban, a dolgozatba csak a szemléletes részeket illesztem be.

A bemutatott algoritmusokat random számokkal feltöltött listákra futtattam le az alábbi módon.
A bemenő listák elemszámát 1000-től indulva, ezrenként lépve, 10000-ig emeltem.
Minden elemszámra több futtatást végeztem.
A metszet és unió algoritmusok esetében ez a szám, a futás hossúsága miatt 10 (az unió 20 perc alatt futott le minden inputra), a többi algoritmusnál felemeltem 100-ra.
Minden futtatásnál új random bemenetet generáltam.
Ahol két bemenő lista van, ott mindkettőnek változtattam az elemszámát, de, szintén a hosszú futásidő miatt, mindig úgy, hogy az első sorozat elemszáma kisebb vagy egyenlő a másodikéval.
Ez aggályos lehet, mivel sem a metszet, sem az unió algoritmusa nem szimmetrikus a paraméterekre nézve.

## 2. Alap algoritmusok

Ebben a részben a metszet, az unió és az összefuttatás algoritmusokat mutatom be.
Ezek közös jellemzője, hogy két sorozatot fogadnak paraméterként, és egy sorozattal térnek vissza.

A metszettel és az unióval kapcsolatban megjegyzem, hogy bár a nevüket ezek az algoritmusok a megfelelő halmazelméleti műveletekről kapták, az alábbi implementációk kihasználják a két bemenő sorozat indexelhetőségét, ezért ezek nem lehetnek halmaz (`set`) típusúak a programban.

Mindkettő implementációjához használtam azt az algoritmust, aminek a segítségével eldönthető, hogy egy sorozat tartalmaz-e egy elemet:

In [2]:
def _does_include(array, element):
    i = 0
    while i < len(array) and element != array[i]:
        i += 1
    return i < len(array)

### 2. 1. Metszet

Az alábbi egyszerű metszet algoritmus végigiterál az első paraméteren (`a`), és mindegyik elemnél megvizsgálja, hogy a másik sorozat (`b`) tartalmazza-e azt az elemet.
Ha igen, az adott elem bekerül a metszetbe.
Mivel a tartalmazás eldöntéséhez végig kell iterálni a `b`-n, az algoritmus egymásba ágyazott ciklusokat tartalmaz, 

In [ ]:
def intersection(array1, array2):
    result = []
    for element in array1:
        if _does_include(array2, element):
            result.append(element)
    return result